# Intro & Loading 

In [ ]:
from __future__ import print_function
import pandas as pd

import os 

from genepy.utils import helper as h
from depmapomics import terra as myterra
from depmapomics.config import *
from gsheets import Sheets
from taigapy import TaigaClient
from bokeh.plotting import output_notebook

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

sheets = Sheets.from_files('~/.client_secrets.json', '~/.storage.json')

virtual = VIRTUAL

In [ ]:
VIRTUAL

In [ ]:
PREV_VIRTUAL

## making a tentative virtual

## making the virtuals

In [ ]:
new = {}
gsheets = sheets.get(POTENTIAL_LIST).sheets[0].to_frame()
new['internal'] = set([i for i in gsheets['Internal'].values.tolist() if str(i) != "nan"])
new['dmc'] = set([i for i in gsheets['DMC'].values.tolist() if str(i) != "nan"])
new['ibm'] = set([i for i in gsheets['IBM'].values.tolist() if str(i) != "nan"])
new['public'] = set([i for i in gsheets['Public'].values.tolist() if str(i) != "nan"])


new["internal"] = new["internal"] | new["ibm"] | new["dmc"] | new["public"]
new["ibm"] = new["ibm"] | new["dmc"] | new["public"]
new["dmc"] = new["dmc"] | new["public"]

## Getting what was released before

In [ ]:
prevmut = {}
prevrna = {}
prevcn = {}
prevwes = {}
prev = {}
for val in datasets:
    print(val)
    prevmut[val] = set(tc.get(name=PREV_VIRTUAL[val], file='CCLE_mutations').DepMap_ID)
    prevrna[val] = set(tc.get(name=PREV_VIRTUAL[val], file='CCLE_expression').index)
    prevcn[val] = set(tc.get(name=PREV_VIRTUAL[val], file='CCLE_segment_cn').DepMap_ID)
    prev[val] = prevmut[val] | prevrna[val] | prevcn[val]
    prevwes[val] = prevmut[val] | prevcn[val]

In [ ]:
prevmut["dmc"] = prevmut["dmc"] | prevmut["public"]
prevrna["dmc"] = prevrna["dmc"] | prevrna["public"]
prevcn["dmc"] = prevcn["dmc"] | prevcn["public"]
prev["dmc"] = prev["dmc"] | prev["public"]
prevwes["dmc"] = prevwes["dmc"] | prevwes["public"]

prevmut["ibm"] = prevmut["ibm"] | prevmut["dmc"]
prevrna["ibm"] = prevrna["ibm"] | prevrna["dmc"]
prevcn["ibm"] = prevcn["ibm"] | prevcn["dmc"]
prev["ibm"] = prev["ibm"] | prev["dmc"]
prevwes["ibm"] = prevwes["ibm"] | prevwes["dmc"]

prevmut["internal"] = prevmut["internal"] | prevmut["ibm"]
prevrna["internal"] = prevrna["internal"] | prevrna["ibm"]
prevcn["internal"] = prevcn["internal"] | prevcn["ibm"]
prev["internal"] = prev["internal"] | prev["ibm"]
prevwes["internal"] = prevwes["internal"] | prevwes["ibm"]

In [ ]:
print('in cn but not mut')
print(prevwes["internal"] - prevmut["internal"])
print('in mut but not cn')
print(prevwes["internal"] - prevcn["internal"])
print('in rna but no wes/wgs')
print(prev["internal"] - prevwes["internal"])
print('in wes/wgs but not rna')
print(prev["internal"] - prevrna["internal"])

## managing the readmes

In [ ]:
#! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

In [ ]:
! cd ../depmap-release-readmes && git pull --no-commit

In [ ]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $RELEASE  && git add . && git commit -m $RELEASE && git push 

In [ ]:
# NOW UPDATE THE READMEs

In [ ]:
import yaml
folder = "../depmap-release-readmes/"
a = ! ls $folder
print(a)
for val in a:
    with open(folder+val,'r+') as file:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
        dict_file = yaml.load(file, Loader=yaml.FullLoader)
        dict_file['extra_sections'][0]['description'] = changes
        yaml.dump(dict_file, file)

In [ ]:
# and save them
! cd ../depmap-release-readmes && cp release-$RELEASE/* ../ccle_processing/readmes/ && git add . && git commit -m "Omics: updating readmes to new release" && git push

# Copy Number

In [ ]:
genecn= pd.read_csv('temp/'+SAMPLESETNAME+'/achilles_gene_cn.csv',index_col=0)
segmentcn = pd.read_csv('temp/'+SAMPLESETNAME+'/achilles_segment.csv')

In [ ]:
#genecn = genecn.apply(lambda x: (x**2)-1)

In [ ]:
virtual= {}
INFO = {}
blacklist = set()

for val in datasets:
    print('_________________________________________________')
    print(val)
    print('removed')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    missing = set(new[val]) - set(segmentcn.DepMap_ID)
    blacklist = (set(segmentcn.DepMap_ID) - (prevcn[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## for segment removing first blacklisted, then embargoed, to create two datasets
    print(len(segmentcn))
    a = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
    print(len(segmentcn) - len(a))
    a.to_csv('temp/all_merged_segments.csv', index=False)
    print(len(genecn))
    a = genecn[~genecn.index.isin(blacklist)]
    print(len(genecn) - len(a))
    a.to_csv('temp/all_merged_genes_cn.csv')
    
    INFO[val] = "# " + val + """ dataset:
                
## DNAseq Omics:

NEW LINES:
"""+str(newlines)+"""

BLACKLIST:
"""+str(blacklist)+"""

MISSING:
"""+str(missing)+"""

REMOVED:
"""+str(removed)
    
    # Add to Taiga 
    # 1.(replace create_dataset() by update_dataset() if the dataset already exists)
    # 2.remove folder_id=virtual_folder
    # 3.add a changes_description="something",
    # 4.add add_all_existing_files=True,
    # 5.replace val+"_"+samplesetname by the virtual dataset id
    virtual[val] = tc.create_dataset(val+"_"+SAMPLESETNAME, 
                    dataset_description = SAMPLESETNAME + " relese of the DepMap dataset for the DepMap Portal. Please look at the README file for additional information about this dataset. ",
                    upload_files=[
                        {
                            "path": "temp/all_merged_genes_cn.csv",
                            "name": "CCLE_gene_cn",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/all_merged_segments.csv",
                            "name": "CCLE_segment_cn",
                            "format": "TableCSV",
                            "encoding": "utf-8"
                        },
                        #{
                        #    "path": 'readmes/'+val+'-'+release+'.yaml',
                        #    "name": "README",
                        #    "format": "Raw",
                        #    "encoding": "utf-8"
                        #}
                    ],
                    #add_all_existing_files=True,
                    #changes_description="rerunning the CN upload script",
                    folder_id=VIRTUAL_FOLDER)

## current dataset

In [ ]:
virtual

# Mutations

## Somatic

In [ ]:
mutations = pd.read_csv("temp/"+SAMPLESETNAME+"/merged_somatic_mutations_withlegacy.csv")
damaging = pd.read_csv('temp/'+SAMPLESETNAME+'/merged_somatic_mutations_boolmatrix_fordepmap_damaging.csv', index_col=0)
othercons = pd.read_csv('temp/'+SAMPLESETNAME+'/merged_somatic_mutations_boolmatrix_fordepmap_othercons.csv', index_col=0)
othernoncons = pd.read_csv('temp/'+SAMPLESETNAME+'/merged_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv', index_col=0)
hotspot = pd.read_csv('temp/'+SAMPLESETNAME+'/merged_somatic_mutations_boolmatrix_fordepmap_hotspot.csv', index_col=0)

In [ ]:
#mutations = mutations.drop(columns='Unnamed: 0')

In [ ]:
blacklist = set()
for val in datasets:
    print('_________________________________________________')
    print(val)
    missing = set(new[val]) - set(mutations.DepMap_ID)
    print('not present')
    removed = set(prev[val]) - set(mutations.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevmut[val]) - set(mutations.DepMap_ID)
    print(removed)
    blacklist = (set(mutations.DepMap_ID) - (prevmut[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(blacklist)
    # adding files
    a = mutations[~mutations.DepMap_ID.isin(blacklist)]
    print(len(mutations)- len(a))
    a.to_csv('temp/all_somatic_mutations_withlegacy.csv', index=False)
    a = damaging[~damaging.index.isin(blacklist)]
    print(len(damaging) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv')
    a = othercons[~othercons.index.isin(blacklist)]
    print(len(othercons) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv')
    a = othernoncons[~othernoncons.index.isin(blacklist)]
    print(len(othernoncons) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv')
    a = hotspot[~hotspot.index.isin(blacklist)]
    print(len(hotspot) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv')
    
    # updating on taiga
    tc.update_dataset(dataset_id=virtual[val],
                      changes_description='adding mutations',
                      upload_files=[
                        {
                            "path": "temp/all_somatic_mutations_withlegacy.csv",
                            "name": "CCLE_mutations",
                            "format": "TableCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv",
                            "name": "CCLE_mutations_bool_damaging",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv",
                            "name": "CCLE_mutations_bool_nonconserving",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv",
                            "name": "CCLE_mutations_bool_otherconserving",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv",
                            "name": "CCLE_mutations_bool_hotspot",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        }
                      ],
                      add_all_existing_files=True)

# RNA

In [ ]:
transcripts_tpm = pd.read_csv('temp/expression_' + SAMPLESETNAME + '_transcripts_tpm_logp1.csv',index_col=0)
genes_tpm = pd.read_csv('temp/expression_' + SAMPLESETNAME + '_genes_tpm_logp1.csv',index_col=0)
genes_expected_count = pd.read_csv('temp/expression_' + SAMPLESETNAME + '_genes_expected_count.csv',index_col=0)
proteincoding_genes_expected_count = pd.read_csv('temp/expression_' + SAMPLESETNAME + '_proteincoding_genes_expected_count.csv',index_col=0)
proteincoding_genes_tpm = pd.read_csv('temp/expression_' + SAMPLESETNAME + '_proteincoding_genes_tpm_logp1.csv',index_col=0)
transcripts_expected_count = pd.read_csv('temp/expression_' + SAMPLESETNAME + '_transcripts_expected_count.csv',index_col=0)
enrichments = pd.read_csv('temp/gene_sets_'+SAMPLESETNAME+'_all.csv', index_col=0)

In [ ]:
%store -r rnafailed

In [ ]:
#blacklist = set()
for val in datasets:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(genes_tpm.index)
    print(removed)
    print('removed for QC reasons')
    print(rnafailed)
    print('removed')
    removed = set(prevrna[val]) - set(genes_tpm.index)
    #print(removed - set(rename.keys()))
    missing = set(new[val]) - set(genes_tpm.index)
    blacklist = (set(genes_tpm.index) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)

    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(genes_expected_count))
    a = genes_expected_count[~genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_expected_count.csv')
    print(len(genes_tpm))
    a = genes_tpm[~genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_tpm.csv')
    print(len(proteincoding_genes_tpm))
    a = proteincoding_genes_tpm[~proteincoding_genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_tpm.csv')
    print(len(transcripts_tpm))
    a = transcripts_tpm[~transcripts_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_tpm.csv')
    print(len(proteincoding_genes_expected_count))
    a = proteincoding_genes_expected_count[~proteincoding_genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_expected_count.csv')
    print(len(transcripts_expected_count))
    a = transcripts_expected_count[~transcripts_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_expected_count.csv')
    #print(len(enrichments))
    #a = enrichments[~enrichments.index.isin(blacklist)]
    #print(len(a))
    #a.to_csv('temp/enrichments_ssGSEA.csv')
    
    INFO[val] += """


## RNAseq Omics:

NEW LINES:
"""+str(newlines)+"""

BLACKLIST:
"""+str(blacklist)+"""

MISSING:
"""+str(missing)+"""

REMOVED:
"""+str(removed)+"""
                
REMOVED FOR QC REASONS:
"""+str(rnafailed)
    
    # adding to taiga
    tc.update_dataset(virtual[val],
                      changes_description='adding expression',
                      upload_files=[
                        {
                            "path": "temp/expression_genes_expected_count.csv",
                            "name": "CCLE_RNAseq_reads",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/expression_transcripts_tpm.csv",
                            "name": "CCLE_RNAseq_transcripts",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/expression_genes_tpm.csv",
                            "name": "CCLE_expression_full",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/expression_proteincoding_genes_tpm.csv",
                            "name": "CCLE_expression",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/expression_proteincoding_genes_expected_count.csv",
                            "name": "CCLE_expression_proteincoding_genes_expected_count",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/expression_transcripts_expected_count.csv",
                            "name": "CCLE_expression_transcripts_expected_count",
                            "format": "NumericMatrixCSV",
                            "encoding": "utf-8"
                        },
                        #{
                        #    "path": "temp/enrichments_ssGSEA.csv",
                        #    "name": "CCLE_ssGSEA",
                        #    "format": "NumericMatrixCSV",
                        #    "encoding": "utf-8"
                        #},
                    ],
                    upload_async=False,
                    add_all_existing_files=True)

# Fusions

In [ ]:
fusions = pd.read_csv('temp/'+SAMPLESETNAME+'/fusions_latest.csv')
filtered = pd.read_csv('temp/'+SAMPLESETNAME+'/filtered_fusions_latest.csv')

In [ ]:
blacklist = set()
for val in datasets:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(fusions.DepMap_ID)
    print(removed)
    print('removed for QC reasons')
    print(failed)
    print('removed')
    removed = set(prevrna[val]) - set(fusions.DepMap_ID)
    print(removed)
    missing = set(new[val]) - set(fusions.DepMap_ID)
    blacklist = (set(fusions.DepMap_ID) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(fusions))
    a = fusions[~fusions.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/fusions.csv', index=False)
    print(len(filtered))
    a= filtered[~filtered.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/filtered_fusions.csv', index=False)

    # uploading to taiga
    tc.update_dataset(virtual[val],
                      changes_description='adding fusions',
                      upload_files=[
                        {
                            "path": "temp/fusions.csv",
                            "name": "CCLE_fusions_unfiltered",
                            "format": "TableCSV",
                            "encoding": "utf-8"
                        },
                        {
                            "path": "temp/filtered_fusions.csv",
                            "name": "CCLE_fusions",
                            "format": "TableCSV",
                            "encoding": "utf-8"
                        },
                      ],
                      dataset_description=INFO[val],
                      add_all_existing_files=True)

# Updating eternal

In [ ]:
def findLatestVersion(dataset, approved_only=True):
    highest = 0
    latest_version = 0
    data = tc.get_dataset_metadata(dataset)
    for val in data['versions']:
        if val['state']=="approved" or not approved_only:
            if int(val['name'])>highest:
                highest = int(val['name'])
                latest_version = highest
    if latest_version==0:
        raise ValueError('could not find a version')
    return data['permanames'][0]+'.'+str(latest_version)

In [ ]:
eternal_dataset = 'depmap-a0ab'

In [ ]:
# To add to a eternal dataset
latest_version = findLatestVersion(virtual['internal'])

files = ["CCLE_gene_cn", "CCLE_segment_cn", 
         
         "CCLE_mutations", "CCLE_mutations_bool_damaging", "CCLE_mutations_bool_nonconserving", "CCLE_mutations_bool_otherconserving", "CCLE_mutations_bool_hotspot", 
         
         "CCLE_expression_full", "CCLE_RNAseq_transcripts", "CCLE_RNAseq_reads", "CCLE_expression", "CCLE_expression_proteincoding_genes_expected_count", "CCLE_expression_transcripts_expected_count",

         "CCLE_fusions_unfiltered", "CCLE_fusions"]

tc.update_dataset(eternal_dataset,
                changes_description='new '+SAMPLESETNAME+" omics dataset.",
                add_taiga_ids=[{"taiga_id": latest_version +"/"+ file, "name": file} for file in files],
                add_all_existing_files=True)

# saving the current release version

In [ ]:
! cd ../ccle_processing && git add . && git commit -m "depmap omics $samplesetname final" && git push

In [ ]:
ls 

In [ ]:
x